In [1]:
import random
import numpy as np

def Pmatrix(n):
    slots=[i for i in range(n)]
    P=matrix(F, n, [0 for i in range(n^2)])
    for i in range(n):
        index=random.choice(slots)
        slots.remove(index)
        P[i,index]=1
    return P

def PmatrixW2(n):
    slots1=[i for i in range(n)]
    slots2=[i for i in range(n)]
    P=matrix(F, n, [0 for i in range(n^2)])

    for i in range(n):
        index1=random.choice(slots1)
        slots1.remove(index1)
        index2=index1
        while index2==index1:
            index2=random.choice(slots2)
        slots2.remove(index2)
        P[i,index1]=1
        P[i,index2]=1
    return P



def Schur_product(M,k1):
    Schur=[]
    for i in range(k1):
        a = np.array(list(M[i]))
        for j in range(i,k1):
            b = np.array(list(M[j]))
            Schur.append(list(np.multiply(a,b)))
    return Schur


def dimrandomcode(n,k):
    if k*(k+1)/2>=n:
        return n
    else:
        return (k*(k+1))/2

def Prob(n,k):
    c1=1
    c2=1
    c=1
    if ((k*(k+1))/2>=n and n>=k):
        return '1 Pr('+str(n)+')>='+str(round(1-2**((-c1)*k)-2**((-c2)*((k*(k-1))/2-n)),10))
    if n>=(k*(k+1)/2):
        return '2 Pr('+str((k*(k+1))/2)+')>='+str(round(1-2**((-c)*(n-(k*(k-1))/2)),10))
    else:
        return 'условие не вписывается в теорему '+'n='+str(n)+' k='+str(k)+' (k*(k+1))/2='+str((k*(k+1))/2)
    
def codegen(k,q,d,codetype,curv=0): #возвращает матрицу кода, функциональное поле, размерность кода,и случайную матрицу
    if codetype in [1,2,3]:
        if codetype==1:
            k.<a> = GF(q^2)
            A.<x,y> = AffineSpace(k, 2)
            C = Curve(y^q + y - x^(q+1))
            Q, = C.places_at_infinity()
        if codetype == 2:
            if curv==0:
                raise Exception('не введена кривая!')
            C = Curve(curv)
            Q, = C.places_at_infinity()
        if codetype == 3:
            A.<x,y> = AffineSpace(k, 2)
            C = Curve(x^3*y+y^3+x)
            p = C([0,0])
            Q, = p.places()

        F = C.function_field()
        pls = F.places()
        pls.remove(Q)
        G = d*Q
        code = codes.EvaluationAGCode(pls, G)  
        print(code)
        M=code.generator_matrix() 
        k1=M.rank()
        S = random_matrix(k,k1,k1) #рандомная матрица
        n=len(M[0])
        while S.is_singular():
            S = random_matrix(k,k1,k1)
        return M,F,n,k1,S
    else:
        raise Exception('не верный тип кода!')

def SchurCalc(n,k1): #вычисление квадратов шура при маскировке весом 1 и 2
    #Вычисление размерности случайного кода
    random_Schur=dimrandomcode(n,k1)

    print()
    print('Размерность квадрата шура случайного кода той же размерности = ', random_Schur)


    P=PmatrixW2(n) #маскировочная матрица веса 2
    P1=Pmatrix(n)
    G_cap_hidden1=Matrix(S*M*P1)
    G_cap_hidden2=Matrix(S*M*P)


    G_cap_Schur1=Matrix(F,Schur_product(G_cap_hidden1,k1))
    G_cap_Schur2=Matrix(F,Schur_product(G_cap_hidden2,k1))

    G1k=G_cap_Schur1.rank()
    G2k=G_cap_Schur2.rank()

    print('Размерность квадрата шура маскировка весом 1', G1k)
    print('Размерность квадрата шура маскировка весом 2', G2k)

    print()
    print('Вычисление вероятности для квадрата шура случайного кода той же размеронсти:', Prob(n,k1))

In [7]:
#Работа с АГ кодами
#задание поля, в случае эрмитовой учитывать что оно будет возведено в квадрат!     
q=4
d=20 #переменная в дивизоре G
'''
Типы кодов:
    1 - эрмитов код, не требует задачи кривой
    2 - эллиптический код, требует задания кривой
    3 - код на квартике клейна, не требует задания кривой
    M,F,k1,S=codegenAG(k,q,d,codetype) эрмитова кривая/квартика
    M,F,n,k1,S=codegenAG(k,q,d,codetype,curv) эллиптическая кривая
'''
codetype=1
k.<a> = GF(q)
A.<x,y> = AffineSpace(k, 2)    
curv=y^2-x^3-7*x-1 #кривая
M,F,n,k1,S=codegen(k,q,d,codetype)
SchurCalc(n,k1)

[64, 15] evaluation AG code over GF(16)

Размерность квадрата шура случайного кода той же размерности =  64
Размерность квадрата шура маскировка весом 1 35
Размерность квадрата шура маскировка весом 2 64

Вычисление вероятности для квадрата шура случайного кода той же размеронсти: 1 Pr(64)>=0.9999694824


In [8]:
#Работа с кодами Рида-Соломона
k = GF(25)
F = k
n, k123 = 24, 6
code = codes.ReedSolomonCode(k, n, k123)
#code = codes.GeneralizedReedSolomonCode(k.list()[:n], k123)
print(code)
M=code.generator_matrix() 
k1=M.rank()
n=len(M[0])
S = random_matrix(k,k1,k1) #рандомная матричка
while S.is_singular():
    S = random_matrix(k,k1,k1)
SchurCalc(n,k1)

[24, 6, 19] Reed-Solomon Code over GF(25)

Размерность квадрата шура случайного кода той же размерности =  21
Размерность квадрата шура маскировка весом 1 11
Размерность квадрата шура маскировка весом 2 21

Вычисление вероятности для квадрата шура случайного кода той же размеронсти: 2 Pr(21)>=0.998046875
